**Task 2**

Instead of the plain inline code, create a function that takes any single band raster path and a shapefile path as input arguments. The function should return a dataframe that calculates the mean and standard deviation of all the polygon features in the provided shapefile path. The input shapefile must have a unique ID column which can be later joined with the output dataframe. For this task, you will find two datasets, i.e., ndvi.tif which shows the NDVI values as single band and grid.geojson as the vector or zones. Look at the next cell to get ideas about how the function might look like. (4.5 pts)



## **Step 1: Mount Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Step 2: Import Necessary Libraries**

In [3]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 56.9 MB/s eta 0:00:00


In [4]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd

## **Step 3: Checking the Content of the Vector File**

In [5]:
# Paths to the data (update these paths based on your directory structure)
raster_path = "/content/drive/MyDrive/Fall2024/GIS5120/Lab6/lab-6-task-data/lab-6-task-data/ndvi.tif"
vector_path = "/content/drive/MyDrive/Fall2024/GIS5120/Lab6/lab-6-task-data/lab-6-task-data/grid.geojson"

# Load vector data (GeoJSON file)
vector_data = gpd.read_file(vector_path)
print("Vector Data Loaded:")
print(vector_data.head())

Vector Data Loaded:
   id           left        top          right     bottom  row_index  \
0  41  583445.254038  4258815.0  594992.259422  4248815.0         16   
1  42  583445.254038  4248815.0  594992.259422  4238815.0         17   
2  61  592105.508076  4303815.0  603652.513459  4293815.0         12   
3  62  592105.508076  4293815.0  603652.513459  4283815.0         13   
4  63  592105.508076  4283815.0  603652.513459  4273815.0         14   

   col_index                                           geometry  
0          1  POLYGON ((583445.254 4253815, 586332.005 42588...  
1          1  POLYGON ((583445.254 4243815, 586332.005 42488...  
2          2  POLYGON ((592105.508 4298815, 594992.259 43038...  
3          2  POLYGON ((592105.508 4288815, 594992.259 42938...  
4          2  POLYGON ((592105.508 4278815, 594992.259 42838...  


## **Step 4: Define the Function for Zonal Statistics**

This function calculates the mean and standard deviation of raster values for each polygon in the vector dataset.

In [6]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd

def zonal_statistics(raster_path, vector_path):
    """
    Compute the mean and standard deviation of raster values for each polygon.

    Parameters:
        raster_path (str): Path to the single-band raster file.
        vector_path (str): Path to the vector file (GeoJSON).

    Returns:
        pd.DataFrame: A DataFrame with statistics for each polygon.
    """
    # Load vector data
    vector_data = gpd.read_file(vector_path)
    print(f"Vector CRS: {vector_data.crs}")

    # Open the raster file
    with rasterio.open(raster_path) as raster:
        print(f"Raster CRS: {raster.crs}")

        # Reproject vector data to match raster CRS if needed
        if vector_data.crs != raster.crs:
            print("Reprojecting vector data to match raster CRS.")
            vector_data = vector_data.to_crs(raster.crs)

        # Initialize storage for statistics
        polygon_ids = []
        mean_values = []
        std_dev_values = []

        # Process each polygon
        for index, polygon in vector_data.iterrows():
            geometry = [polygon['geometry']]
            polygon_id = polygon['id']
            try:
                # Mask raster data using polygon
                masked, _ = rasterio.mask.mask(raster, geometry, crop=True)
                masked_data = masked[0]

                # Filter out no-data values
                valid_data = masked_data[masked_data != raster.nodata]

                # Compute statistics
                if valid_data.size > 0:
                    mean_values.append(valid_data.mean())
                    std_dev_values.append(valid_data.std())
                else:
                    mean_values.append(np.nan)
                    std_dev_values.append(np.nan)

                polygon_ids.append(polygon_id)

            except Exception as e:
                print(f"Error processing polygon ID {polygon_id}: {e}")
                polygon_ids.append(polygon_id)
                mean_values.append(np.nan)
                std_dev_values.append(np.nan)

    # Compile results into a DataFrame
    results = pd.DataFrame({
        'Polygon_ID': polygon_ids,
        'Mean_Value': mean_values,
        'Std_Dev': std_dev_values
    })

    print("Zonal Statistics computation completed.")
    return results

results = zonal_statistics(raster_path, vector_path)

Vector CRS: EPSG:32615
Raster CRS: EPSG:32615
Zonal Statistics computation completed.


In [7]:
# Display the first few rows of the resulting DataFrame
print("Zonal Statistics:")
results.head()

Zonal Statistics:


,Polygon_ID,Mean_Value,Std_Dev
0,41,0.405855,0.067818
1,42,0.406319,0.058588
2,61,0.424627,0.065736
3,62,0.417399,0.071759
4,63,0.363553,0.140118


# **Step 5: Merge Zonal Statistics with the Original Vector Data**

In [8]:
# Load the original vector data again
vector_data = gpd.read_file(vector_path)

# Merge the results DataFrame with the vector data using the unique ID column
merged_data = vector_data.merge(results, left_on='id', right_on='Polygon_ID')

# Display the first few rows of the merged dataset
print("Merged Dataset:")
merged_data.head()

Merged Dataset:


,id,left,top,right,bottom,row_index,col_index,geometry,Polygon_ID,Mean_Value,Std_Dev
0,41,583445.254038,4258815.0,594992.259422,4248815.0,16,1,"POLYGON ((583445.254 4253815, 586332.005 42588...",41,0.405855,0.067818
1,42,583445.254038,4248815.0,594992.259422,4238815.0,17,1,"POLYGON ((583445.254 4243815, 586332.005 42488...",42,0.406319,0.058588
2,61,592105.508076,4303815.0,603652.513459,4293815.0,12,2,"POLYGON ((592105.508 4298815, 594992.259 43038...",61,0.424627,0.065736
3,62,592105.508076,4293815.0,603652.513459,4283815.0,13,2,"POLYGON ((592105.508 4288815, 594992.259 42938...",62,0.417399,0.071759
4,63,592105.508076,4283815.0,603652.513459,4273815.0,14,2,"POLYGON ((592105.508 4278815, 594992.259 42838...",63,0.363553,0.140118


## **Step 6: Save the Final Output as a Shapefile**

In [9]:
# Define the output path for the shapefile
output_shapefile_path = "/content/drive/MyDrive/Fall2024/GIS5120/Lab6/lab-6-task-data/lab-6-task-data/out/final_vector_with_stats.shp"

# Save the merged dataset as a shapefile
merged_data.to_file(output_shapefile_path)

print(f"Merged data with statistics saved to: {output_shapefile_path}")

Merged data with statistics saved to: /content/drive/MyDrive/Fall2024/GIS5120/Lab6/lab-6-task-data/lab-6-task-data/out/final_vector_with_stats.shp
